In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import librosa
import librosa.display
import IPython.display as ipd
import matplotlib.pyplot as plt
from tqdm import tqdm
from keras.models import Sequential
from keras.layers import Dense, Conv2D,Flatten,MaxPooling2D
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping
from sklearn import metrics
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime
import os
import glob
import random

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
def generate_csv(folder, label):
    folder_name = os.path.basename(folder)
    labels = list(label)
    gender_folder={'Nam','Nu'}
    genders = list(gender_folder)
    print(labels)
    # generate CSV file
    df = pd.DataFrame(columns=["filepath", "label"])
    i = 0
    for gender in genders:
     for label in labels:
        print("Reading", os.path.join(folder, gender, label, "*"))
        for filepath in glob.glob(os.path.join(folder, gender, label, "*","*.wav")):
            df.loc[i] = [filepath,label]
            i += 1
    output_file = f"{folder_name}.csv"
    print("Saving", output_file)
    df.to_csv(output_file)
# dir_folder = '/content/gdrive/MyDrive/DATK/Training/Number'
dir_folder = '/content/gdrive/MyDrive/DATK/Check/Train'
generate_csv(dir_folder, {'zero','one','two','three','four','five','six','seven','eight','nine'})

['four', 'three', 'five', 'zero', 'one', 'seven', 'six', 'nine', 'eight', 'two']
Reading /content/gdrive/MyDrive/DATK/Check/Train/Nam/four/*
Reading /content/gdrive/MyDrive/DATK/Check/Train/Nam/three/*
Reading /content/gdrive/MyDrive/DATK/Check/Train/Nam/five/*
Reading /content/gdrive/MyDrive/DATK/Check/Train/Nam/zero/*
Reading /content/gdrive/MyDrive/DATK/Check/Train/Nam/one/*
Reading /content/gdrive/MyDrive/DATK/Check/Train/Nam/seven/*
Reading /content/gdrive/MyDrive/DATK/Check/Train/Nam/six/*
Reading /content/gdrive/MyDrive/DATK/Check/Train/Nam/nine/*
Reading /content/gdrive/MyDrive/DATK/Check/Train/Nam/eight/*
Reading /content/gdrive/MyDrive/DATK/Check/Train/Nam/two/*
Reading /content/gdrive/MyDrive/DATK/Check/Train/Nu/four/*
Reading /content/gdrive/MyDrive/DATK/Check/Train/Nu/three/*
Reading /content/gdrive/MyDrive/DATK/Check/Train/Nu/five/*
Reading /content/gdrive/MyDrive/DATK/Check/Train/Nu/zero/*
Reading /content/gdrive/MyDrive/DATK/Check/Train/Nu/one/*
Reading /content/gdrive/

In [ ]:
# metadata_filename = "Number.csv"
metadata_filename = "Train.csv"
# metadata_filename = "Check.csv"
# load CSV files as DataFrames
metadata = pd.read_csv(metadata_filename)
metadata.tail()

,Unnamed: 0,filepath,label
19262,19262,/content/gdrive/MyDrive/DATK/Check/Train/Nu/tw...,two
19263,19263,/content/gdrive/MyDrive/DATK/Check/Train/Nu/tw...,two
19264,19264,/content/gdrive/MyDrive/DATK/Check/Train/Nu/tw...,two
19265,19265,/content/gdrive/MyDrive/DATK/Check/Train/Nu/tw...,two
19266,19266,/content/gdrive/MyDrive/DATK/Check/Train/Nu/tw...,two


In [ ]:
# def features_extractor(file_name):
#     audio, sample_rate = librosa.load(file_name)
#     mel = librosa.feature.melspectrogram(y=audio, sr=sample_rate, n_fft=2048, hop_length=512, n_mels=128)
#     mels = np.zeros((128, 44))
#     mels[:mel.shape[0], :mel.shape[1]]=mel
#     #mels = librosa.amplitude_to_db(mels, ref=np.max)
#     mels = np.transpose(mels)
#     return mels

def features_extractor(file_name):
    audio, sample_rate = librosa.load(file_name)

    # Tính toán số frame trong mỗi frame có độ dài 25ms với sampling rate là 16000
    frame_length = int(0.025 * sample_rate)
    # Tính toán số sample overlap trong frame overlap 10ms với sampling rate là 16000
    hop_length = int(0.01 * sample_rate)
    # Tính toán kích thước FFT (Fast Fourier Transform) trong hàm mfcc
    n_fft = 2 ** int(np.ceil(np.log2(frame_length)))

    mfcc = librosa.feature.mfcc(y=audio, sr=sample_rate, n_fft=n_fft, hop_length=hop_length, n_mfcc=13)
    mfccs = np.zeros((13,101))
    mfccs[:mfcc.shape[0], :mfcc.shape[1]]=mfcc
    delta_mfccs = librosa.feature.delta(mfccs)
    delta2_mfccs = librosa.feature.delta(mfccs, order=2)
    mfccs_features = np.concatenate((mfccs, delta_mfccs, delta2_mfccs))
    mfccs_features = np.transpose(mfccs_features)

    return mfccs_features

In [ ]:
extracted_features1=[]
for index_num,row in metadata.iterrows():
    filename = str(row["filepath"])
    final_class_labels=row["label"]
    data=features_extractor(filename)
    extracted_features1.append([data,final_class_labels])

In [ ]:
extracted_features_df=pd.DataFrame(extracted_features1,columns=['feature','class'])
outputfile = f"{'extracted_feature_file'}.csv"
extracted_features_df.to_csv(outputfile)
extracted_features_df.tail()

,feature,class
19262,"[[-498.1611328125, 69.26548767089844, -36.1395...",two
19263,"[[-604.4554443359375, 0.3930078148841858, 0.32...",two
19264,"[[-747.1950073242188, 86.9214859008789, 17.627...",two
19265,"[[-338.83404541015625, 58.518043518066406, -10...",two
19266,"[[-430.4992370605469, 41.178123474121094, -109...",two


In [ ]:
x_data = np.stack(extracted_features_df['feature'])
y_data =np.array(extracted_features_df['class'].tolist())
x_data.shape

(19267, 101, 39)

In [ ]:
labelencoder=LabelEncoder()
y_encoder=to_categorical(labelencoder.fit_transform(y_data))

In [ ]:
def split_data_valid(x_data, y_data, valid_size=0.2):
    # split training set and testing set
    X_train, X_valid, y_train, y_valid = train_test_split(x_data, y_data, test_size=valid_size, random_state=7)
    # return a dictionary of values
    return {
        "X_train": X_train,
        "X_valid": X_valid,
        "y_train": y_train,
        "y_valid": y_valid
    }
data = split_data_valid(x_data, y_encoder, valid_size=0.2)

In [ ]:
from keras.models import load_model
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Bidirectional
from keras import Input
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM,SimpleRNN,GRU
from tensorflow.keras.utils import to_categorical

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay
from tensorflow.keras.callbacks import Callback, ModelCheckpoint

In [ ]:
def get_sequence_model():
    frame_features_input = Input(data["X_train"].shape[1:])
    x = GRU(128, return_sequences=True)(
        frame_features_input)
    x = keras.layers.GRU(64)(x)
    x = keras.layers.Dropout(0.2)(x)
    # x = GRU(128,)(frame_features_input)
    x = keras.layers.Dense(64, activation="relu")(x)
    x = keras.layers.Dropout(0.2)(x)
    # x = keras.layers.Dense(32, activation="relu")(x)
    # x = keras.layers.Dropout(0.2)(x)
    # x = keras.layers.Dense(16, activation="relu")(x)
    # x = keras.layers.Dropout(0.2)(x)

    output = keras.layers.Dense(10, activation="softmax")(x)
    rnn_model = keras.Model(frame_features_input, output)
    return rnn_model
model = get_sequence_model()


# model = Sequential()
# model.add(Flatten(input_shape=(39,44)))
# model.add(Dense(256, activation='relu'))
# model.add(Dropout(0.1))
# model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.1))
# model.add(Dense(64, activation='relu'))
# # model.add(Dropout(0.1))
# model.add(Dense(10, activation='softmax'))

# model = Sequential()
# model.add(Flatten(input_shape=(39,44)))
# model.add(Dense(128,activation ='relu'))
# model.add(Dropout(0.1))
# model.add(Dense(128, activation="relu"))
# model.add(Dropout(0.1))
# model.add(Dense(64, activation="relu"))
# model.add(Dropout(0.1))
# model.add(Dense(64, activation="relu"))
# model.add(Dropout(0.1))
# model.add(Dense(32, activation="relu"))
# model.add(Dense(10, activation='softmax'))

In [ ]:

model.compile( loss='categorical_crossentropy', metrics=[ 'accuracy'],optimizer='adam')
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 101, 39)]         0         
                                                                 
 gru (GRU)                   (None, 101, 128)          64896     
                                                                 
 gru_1 (GRU)                 (None, 64)                37248     
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 10)                650   

In [ ]:
checkpointer = ModelCheckpoint(filepath='/content/gdrive/MyDrive/DATK/Check/GRU_model_17_7.h5', monitor='val_accuracy', verbose=1, save_best_only=True)
# checkpointer = ModelCheckpoint(filepath='saved_models/best_fcn.hdf5', monitor='val_accuracy', verbose=1, save_best_only=True)

callbacks = [checkpointer]
batch_size = 32
epochs = 300
# train the model using the training set and validating using validation set
history = model.fit(data["X_train"], data["y_train"], epochs=epochs, batch_size=batch_size, validation_data=(data["X_valid"], data["y_valid"]),
          callbacks=callbacks)

Epoch 1/300
482/482 [==============================] - ETA: 0s - loss: 1.2301 - accuracy: 0.5630
Epoch 1: val_accuracy improved from -inf to 0.80566, saving model to /content/gdrive/MyDrive/DATK/Check/GRU_model_17_7.h5
482/482 [==============================] - 121s 232ms/step - loss: 1.2301 - accuracy: 0.5630 - val_loss: 0.5968 - val_accuracy: 0.8057
Epoch 2/300
482/482 [==============================] - ETA: 0s - loss: 0.4658 - accuracy: 0.8547
Epoch 2: val_accuracy improved from 0.80566 to 0.89336, saving model to /content/gdrive/MyDrive/DATK/Check/GRU_model_17_7.h5
482/482 [==============================] - 112s 232ms/step - loss: 0.4658 - accuracy: 0.8547 - val_loss: 0.3209 - val_accuracy: 0.8934
Epoch 3/300
482/482 [==============================] - ETA: 0s - loss: 0.3300 - accuracy: 0.8988
Epoch 3: val_accuracy did not improve from 0.89336
482/482 [==============================] - 104s 215ms/step - loss: 0.3300 - accuracy: 0.8988 - val_loss: 0.3945 - val_accuracy: 0.8778
Epoch 

KeyboardInterrupt: ignored

In [ ]:
best_model = load_model('/content/gdrive/MyDrive/DATK/Check/GRU_model_16_7.h5')

OSError: ignored

In [ ]:
print(f"Evaluating the model using {len(data['X_valid'])} samples...")
loss, accuracy = best_model.evaluate(data["X_valid"], data["y_valid"], verbose=0)
print(f"Loss: {loss:.4f}")
print(f"Accuracy: {accuracy*100:.2f}%")

In [ ]:
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'valid'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'valid'], loc='upper left')
plt.show()